In [1]:
from openai import OpenAI
import os

In [4]:
api_key = os.environ.get('openapi_key')

In [1]:
client = OpenAI(api_key=api_key)

In [3]:

completion = client.completions.create(model='davinci-002',
                                       prompt="gimme one reason to learn how to work with openai api",
                                       max_tokens=40)
#print(completion.choices[0].text)